## Dependencies

Due to this codebase being inherited from 2022, we have to install a previous version of torch due to torchtext being deprecated.

In [ ]:
!pip install --upgrade torch==2.3.0
!pip install --upgrade torchtext==0.18.0
!pip install --upgrade torchvision==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 827.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Shared codebase

The following defines useful methods, the definition of models to use and anything that is not subject to change.*italicized text*

In [ ]:
import torch


import time
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab as torch_vocab
from torch.utils.data import DataLoader


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from sklearn.metrics import f1_score, precision_score, recall_score

def build_vocab_from_corpus(dataset, tokenizer):
    """
    Given a corpus, builds a Vocab object with tokenizer function.
    """
    counter = Counter()
    for _headline, _body, _ in dataset:
        counter.update(tokenizer(_headline + "\n\n" + _body))
    v = torch_vocab(counter, specials=["BOH", "EOH", "BOP", "EOP"])
    return v

def calculate_metrics(predictions, y_true):
    """
    A simple funcion that given predictions and the expected results
    will return the precision, recall and f1 (macro).
    """
    y_pred = predictions.argmax(dim=1)
    y_true = y_true.argmax(dim=1)
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    return precision, recall, f1

def training_step(model, dataloader, optimizer, criterion, device):
    """
    Given a model, a dataloader, a optimizer and a criterion, it wil perform
    a training step in batches.
    """

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    i = 0

    for labels, headlines, bodies in dataloader:
        i += 1
        # Move to device (usually GPU/TPU)
        labels = labels.to(device)
        headlines = headlines.to(device)
        bodies = bodies.to(device)
        # Reset gradients for previous steps
        optimizer.zero_grad()

        # Perform predictions
        predictions = model(headlines, bodies)

        # Criterion to define target
        loss = criterion(predictions, torch.argmax(labels, dim=1))

        # Metrics
        predictions = predictions.to('cpu')
        labels = labels.to('cpu')
        headlines = headlines.to('cpu')
        bodies = bodies.to('cpu')
        precision, recall, f1 = calculate_metrics(predictions, labels)

        # Gradients
        loss.backward()

        # Optimization
        optimizer.step()

        # Update metrics
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(dataloader), epoch_precision / len(
        dataloader), epoch_recall / len(dataloader), epoch_f1 / len(dataloader)

def evaluate(model, dataloader, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for labels, headlines, bodies in dataloader:

            labels = labels.to(device)
            headlines = headlines.to(device)
            bodies = bodies.to(device)

            # Predecimos
            predictions = model(headlines, bodies)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, torch.argmax(labels, dim=1))

            predictions = predictions.to('cpu')
            labels = labels.to('cpu')
            headlines = headlines.to('cpu')
            bodies = bodies.to('cpu')

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, labels)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(dataloader), epoch_precision / len(
        dataloader), epoch_recall / len(dataloader), epoch_f1 / len(dataloader)


def run_experiment(model, model_name, criterion, train_dataloader, valid_dataloader, n_epochs, device):

  model.to(device)
  criterion.to(device)
  best_valid_loss = float('inf')

  for epoch in range(n_epochs):

      start_time = time.time()

      # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

      # Entrenar
      train_loss, train_precision, train_recall, train_f1 = training_step(
            model, train_dataloader, optimizer, criterion, device)

      print("Finishing training epoch")

        # Evaluar (valid = validación)
      valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
            model, valid_dataloader, criterion)

        #print("Finishing validation epoch")
      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
        # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), '/content/drive/MyDrive/Tesis/{}.pt'.format(model_name))
        # Si ya no mejoramos el loss de validación, terminamos de entrenar.

      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
      print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


class ClickbaitDataset(Dataset):
    """
    Dataset class for Clickbait Webis 20217 Dataset.
    Assumes data has already been preprocessed and has a format
    (headline, body, truthClass).
    """
    def __init__(self, path='data/clickbait', split='train'):
        if split not in ['train', 'test', 'valid']:
            raise ValueError(f'Dataset {split} not found, it must be train, valid or test')
        self.split = split
        if split == 'valid':
            split = 'validation'
        dataset = pd.read_csv(f'{path}/{split}/{split}.csv')
        self.headlines = list(dataset['headline'])
        self.bodies = list(dataset['body'])
        self.labels = list(dataset['truthClass'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.headlines[idx], self.bodies[idx], self.labels[idx]


class IncongruenceDataset(Dataset):
    """
    Dataset for FNC-1 Dataset.
    Assumes data has already been preprocessed and
    has a format (headline, body, labels)
    """

    def __init__(self, path='../data/fnc-1', split='train'):
        if split not in ['train', 'test', 'valid']:
            raise ValueError(f'Dataset {split} not found, it must be train, valid or test')
        self.split= split
        dataset = pd.read_csv(f'{path}/{split}.csv')
        self.headlines = list(dataset['headline'])
        self.bodies = list(dataset['body'])
        self.labels = list(dataset['label'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.headlines[idx], self.bodies[idx], self.labels[idx])


class BiDualEncoder(nn.Module):

    def __init__(
            self, vocab_size,
            vectors=None,
            embed_dim=300,
            hidden_dim=128,
            hidden_layers=2,
            output_dim=2,
            freeze_embed=True,
            dropout=0.3,
        ):

        super().__init__()

        self.hidden_dim = hidden_dim

        # Shared embedding layer
        if vectors is not None:
          self.embedding_layer = nn.Embedding(vocab_size, embed_dim).from_pretrained(
              vectors,
              freeze=freeze_embed
          )
        else:
          self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)

        # Encoders
        self.headline_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=hidden_layers, bidirectional=True)
        self.body_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=hidden_layers, bidirectional=True)

        # Prediction
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, headline, body):

        # Representations
        headline_embedding = self.embedding_layer(headline)
        body_embedding = self.embedding_layer(body)

        # Headline latent representation
        headline_lstm, _ = self.headline_lstm(headline_embedding)

        # Body latent representation
        body_lstm, _ = self.body_lstm(body_embedding)

        # Average latest output (bidirectional)
        # This is performerd taking the last
        output_lstm = (headline_lstm[-1,:,:self.hidden_dim] + body_lstm[-1,:,:self.hidden_dim]) / 2
        output = self.fc(self.dropout(output_lstm))
        return output


class BiConditionalDualEncoder(nn.Module):

    def __init__(
            self, vocab_size,
            vectors=None,
            embed_dim=300,
            hidden_dim=128,
            hidden_layers=2,
            output_dim=2,
            freeze_embed=True,
            dropout=0.3,
        ):

        super().__init__()

        self.hidden_dim = hidden_dim

        # Shared embedding layer
        if vectors is not None:
          self.embedding_layer = nn.Embedding(vocab_size, embed_dim).from_pretrained(
              vectors,
              freeze=freeze_embed
          )
        else:
          self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)

        # Encoders
        self.headline_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=hidden_layers, bidirectional=True, dropout=dropout)
        self.body_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=hidden_layers, bidirectional=True, dropout=dropout)

        # Prediction
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, headline, body):

        # Representations
        headline_embedding = self.embedding_layer(headline)
        body_embedding = self.embedding_layer(body)

        # Headline latent representation
        _, (h_hidden, h_cell) = self.headline_lstm(headline_embedding)

        # Body latent representation
        body_lstm, _ = self.body_lstm(body_embedding, (h_hidden, h_cell))

        # Average outputs of body
        output_lstm = (body_lstm[-1,:,:self.hidden_dim] + body_lstm[-1,:,self.hidden_dim:]) / 2
        output = self.fc(self.dropout(output_lstm))
        return output


class LTSM_Encoder(nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers, padding_idx, vectors, bidirectional=False, dropout=0.3):
        super().__init__()
        # Embedding layer
        self.embed_layer = nn.Embedding(input_dim, embedding_dim).from_pretrained(vectors, freeze=True)
        # LSTM layer
        self.hidden_dim = hidden_dim
        self.lstm_headline = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout if num_layers > 1 else 0)
        self.lstm_body = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout if num_layers > 1 else 0)
        # Output layer
        self.fc = nn.Linear(hidden_dim * 4 if bidirectional else hidden_dim * 2, output_dim)
        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, headline, body):
        # Embed text)
        headline_embed = self.embed_layer(headline)
        body_embed = self.embed_layer(body)
        # LSTM layer
        outputs_heads, (hidden_heads, cell_heads) = self.lstm_headline(headline_embed)
        outputs_bodies, (hidden_bodies, cell_bodies) = self.lstm_body(body_embed)
        outputs_heads = outputs_heads[-1]
        outputs_bodies = outputs_bodies[-1]
        outputs = torch.cat((outputs_heads, outputs_bodies), 1)
        # Make predictions
        predictions = self.fc(self.dropout(outputs))
        return predictions

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

## Tokenization and collation

While ideally this code is subject to little changes, we prefer to separate it from the shared and static codebase.



In [ ]:
import re
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

vocab_transform = lambda x: [vocab[token] if token in vocab else 0 for token in tokenizer(x)]

tensor_transform = lambda x: torch.tensor(x)

def label_transform(label):
    if label == 'clickbait':
        return [1, 0]
    elif label == 'no-clickbait':
        return [0, 1]

def collate_batch(batch):
    articles, headlines, labels = [], [], []
    for (_headline, _body, _label) in batch:
        # Transform the headline
        _headline = tensor_transform(vocab_transform(_headline))
        _body = tensor_transform(vocab_transform(_body)[:300])
        headlines.append(_headline)
        articles.append(_body)
        labels.append(label_transform(_label))
    # Transform labels to tensor and add padding to articles
    labels = torch.tensor(labels)
    headlines = pad_sequence(headlines)
    articles = pad_sequence(articles)

    pad = articles.shape[0] - headlines.shape[0]
    headlines = F.pad(headlines, pad=(0,0, 0, pad), value=0)
    # Transform labels to tensor and add padding to articles
    return labels, headlines, articles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dataset = ClickbaitDataset(path="/content/drive/MyDrive/Tesis/Tesis/clickbait", split="train")
valid_dataset = ClickbaitDataset(path="/content/drive/MyDrive/Tesis/Tesis/clickbait", split="valid")

## Vocab

To count number of tokens, bu we will use Glove with 300 dimensions.

In [ ]:
vocab = build_vocab_from_corpus(train_dataset, tokenizer)
vocab
len(vocab)

182486

In [ ]:
from torchtext.vocab import GloVe
vectors = GloVe(name='6B', dim=300)
vocab = torch_vocab(vectors.stoi, specials=["BOH", "EOH", "BOP", "EOP", "<UNK>"])
vocab.set_default_index(-1)

.vector_cache/glove.6B.zip: 862MB [04:19, 3.32MB/s]                           
100%|█████████▉| 399999/400000 [00:54<00:00, 7378.81it/s]


In [ ]:
train_dataloader = DataLoader(list(train_dataset), batch_size=32, shuffle=True,
                                collate_fn=collate_batch)
valid_dataloader = DataLoader(list(valid_dataset), batch_size=32, shuffle=True,
                                collate_fn=collate_batch)

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 32
OUTPUT_DIM = 2
NUM_LAYERS = 2

model = LTSM_Encoder(input_dim=INPUT_DIM, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM, num_layers=NUM_LAYERS, padding_idx=0, vectors=vectors.vectors)
model_name = "LTSMEncoderGloveClickabitEXP4"

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 5

run_experiment(
    model=model, model_name=model_name,
    criterion=criterion,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    n_epochs=n_epochs,
    device=device
)

Finishing training epoch
Epoch: 01 | Epoch Time: 0m 35s
	Train Loss: 0.561 | Train f1: 0.43 | Train precision: 0.38 | Train recall: 0.50
	 Val. Loss: 0.563 |  Val. f1: 0.43 |  Val. precision: 0.38 | Val. recall: 0.50
Finishing training epoch
Epoch: 02 | Epoch Time: 0m 34s
	Train Loss: 0.554 | Train f1: 0.43 | Train precision: 0.38 | Train recall: 0.50
	 Val. Loss: 0.558 |  Val. f1: 0.43 |  Val. precision: 0.38 | Val. recall: 0.50
Finishing training epoch
Epoch: 03 | Epoch Time: 0m 35s
	Train Loss: 0.543 | Train f1: 0.46 | Train precision: 0.48 | Train recall: 0.52
	 Val. Loss: 0.537 |  Val. f1: 0.50 |  Val. precision: 0.62 | Val. recall: 0.54
Finishing training epoch
Epoch: 04 | Epoch Time: 0m 33s
	Train Loss: 0.524 | Train f1: 0.52 | Train precision: 0.62 | Train recall: 0.55
	 Val. Loss: 0.532 |  Val. f1: 0.53 |  Val. precision: 0.66 | Val. recall: 0.55
Finishing training epoch
Epoch: 05 | Epoch Time: 0m 36s
	Train Loss: 0.503 | Train f1: 0.56 | Train precision: 0.69 | Train recall: 

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 32
OUTPUT_DIM = 2
NUM_LAYERS = 2

model = BiDualEncoder(vocab_size=INPUT_DIM, embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM, hidden_layers=NUM_LAYERS)
model_name = "BiDualEncoderClickbaitEXP1"

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
print(model)

BiDualEncoder(
  (embedding_layer): Embedding(400004, 300)
  (headline_lstm): LSTM(300, 32, num_layers=2, bidirectional=True)
  (body_lstm): LSTM(300, 32, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (softmax): Softmax(dim=0)
)


In [ ]:
n_epochs = 5

run_experiment(
    model=model, model_name=model_name,
    criterion=criterion,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    n_epochs=n_epochs,
    device=device
)

Finishing training epoch
Epoch: 01 | Epoch Time: 0m 56s
	Train Loss: 0.556 | Train f1: 0.43 | Train precision: 0.38 | Train recall: 0.50
	 Val. Loss: 0.549 |  Val. f1: 0.43 |  Val. precision: 0.38 | Val. recall: 0.50
Finishing training epoch
Epoch: 02 | Epoch Time: 0m 55s
	Train Loss: 0.514 | Train f1: 0.50 | Train precision: 0.55 | Train recall: 0.54
	 Val. Loss: 0.543 |  Val. f1: 0.52 |  Val. precision: 0.62 | Val. recall: 0.55
Finishing training epoch
Epoch: 03 | Epoch Time: 0m 56s
	Train Loss: 0.391 | Train f1: 0.74 | Train precision: 0.80 | Train recall: 0.73
	 Val. Loss: 0.598 |  Val. f1: 0.51 |  Val. precision: 0.56 | Val. recall: 0.53
Finishing training epoch
Epoch: 04 | Epoch Time: 0m 56s
	Train Loss: 0.246 | Train f1: 0.86 | Train precision: 0.89 | Train recall: 0.85
	 Val. Loss: 0.748 |  Val. f1: 0.58 |  Val. precision: 0.61 | Val. recall: 0.59
Finishing training epoch
Epoch: 05 | Epoch Time: 0m 57s
	Train Loss: 0.173 | Train f1: 0.90 | Train precision: 0.92 | Train recall: 

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 32
OUTPUT_DIM = 2
NUM_LAYERS = 2

model = BiConditionalDualEncoder(vocab_size=INPUT_DIM, embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM, hidden_layers=NUM_LAYERS)
model_name = "BiConditionalDualEncoderClickbaitEXP3"

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
run_experiment(
    model=model, model_name=model_name,
    criterion=criterion,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    n_epochs=n_epochs,
    device=device
)

Finishing training epoch
Epoch: 01 | Epoch Time: 1m 17s
	Train Loss: 0.517 | Train f1: 0.52 | Train precision: 0.54 | Train recall: 0.55
	 Val. Loss: 0.478 |  Val. f1: 0.66 |  Val. precision: 0.71 | Val. recall: 0.65
Finishing training epoch
Epoch: 02 | Epoch Time: 1m 11s
	Train Loss: 0.399 | Train f1: 0.73 | Train precision: 0.78 | Train recall: 0.72
	 Val. Loss: 0.488 |  Val. f1: 0.67 |  Val. precision: 0.70 | Val. recall: 0.68
Finishing training epoch
Epoch: 03 | Epoch Time: 0m 57s
	Train Loss: 0.244 | Train f1: 0.86 | Train precision: 0.88 | Train recall: 0.86
	 Val. Loss: 0.598 |  Val. f1: 0.67 |  Val. precision: 0.69 | Val. recall: 0.67
Finishing training epoch
Epoch: 04 | Epoch Time: 0m 56s
	Train Loss: 0.116 | Train f1: 0.94 | Train precision: 0.95 | Train recall: 0.94
	 Val. Loss: 0.758 |  Val. f1: 0.65 |  Val. precision: 0.67 | Val. recall: 0.65
Finishing training epoch
Epoch: 05 | Epoch Time: 0m 59s
	Train Loss: 0.065 | Train f1: 0.97 | Train precision: 0.97 | Train recall: 

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 32
OUTPUT_DIM = 2
NUM_LAYERS = 2

model = BiConditionalDualEncoder(vocab_size=INPUT_DIM, embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM, hidden_layers=NUM_LAYERS, vectors=vectors.vectors)
model_name = "BiConditionalDualEncoderGloveClickbaitEXP3"

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 5

run_experiment(
    model=model, model_name=model_name,
    criterion=criterion,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    n_epochs=n_epochs,
    device=device
)

Finishing training epoch
Epoch: 01 | Epoch Time: 0m 48s
	Train Loss: 0.534 | Train f1: 0.46 | Train precision: 0.46 | Train recall: 0.52
	 Val. Loss: 0.503 |  Val. f1: 0.57 |  Val. precision: 0.68 | Val. recall: 0.58
Finishing training epoch
Epoch: 02 | Epoch Time: 0m 42s
	Train Loss: 0.479 | Train f1: 0.62 | Train precision: 0.70 | Train recall: 0.61
	 Val. Loss: 0.492 |  Val. f1: 0.60 |  Val. precision: 0.70 | Val. recall: 0.60
Finishing training epoch
Epoch: 03 | Epoch Time: 0m 41s
	Train Loss: 0.433 | Train f1: 0.68 | Train precision: 0.75 | Train recall: 0.68
	 Val. Loss: 0.496 |  Val. f1: 0.66 |  Val. precision: 0.72 | Val. recall: 0.66
Finishing training epoch
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.376 | Train f1: 0.75 | Train precision: 0.80 | Train recall: 0.74
	 Val. Loss: 0.507 |  Val. f1: 0.65 |  Val. precision: 0.70 | Val. recall: 0.65
Finishing training epoch
Epoch: 05 | Epoch Time: 0m 39s
	Train Loss: 0.308 | Train f1: 0.81 | Train precision: 0.84 | Train recall: 

## BERT

In [ ]:
!pip install transformers &> /dev/null
!pip install sentencepiece &> /dev/null
!pip install torchmetrics & > /dev/null

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 14.9 MB/s eta 0:00:00


BERT comes with its own tokenizer

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

We need to change our collate_batch function to adapt our data to BERT

In [ ]:
label_dict = {
    'clickbait': 0,
    'no-clickbait': 1,
}

def collate_batch(batch):

    token_ids, mask_ids, segment_ids, labels = [], [], [], []

    for (_headline, _body, _label) in batch:
        _headline_id = tokenizer.encode(_headline, add_special_tokens=False)
        _headline_len = len(_headline_id)
        _body_id = tokenizer.encode(_body[:(512 - _headline_len - 3)], add_special_tokens=False)
        _pair_token_ids = [tokenizer.cls_token_id] + _body_id + [tokenizer.sep_token_id] + _headline_id + [tokenizer.sep_token_id]

        _body_len = len(_body_id)

        _segment_ids = torch.tensor([0] * (_body_len + 2) + [1] * (_headline_len + 1))
        _attn_mask_ids = torch.tensor([1] * (_body_len + _headline_len + 3))

        token_ids.append(torch.tensor(_pair_token_ids))
        segment_ids.append(_segment_ids)
        mask_ids.append(_attn_mask_ids)

        labels.append(label_dict[_label])

    token_ids = pad_sequence(token_ids, batch_first=True)
    segment_ids = pad_sequence(segment_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    labels = torch.tensor(labels)

    return token_ids, mask_ids, segment_ids, labels

In [ ]:
train_dataloader = DataLoader(list(train_dataset), batch_size=32, shuffle=True,
                                collate_fn=collate_batch)
valid_dataloader = DataLoader(list(valid_dataset), batch_size=32, shuffle=True,
                                collate_fn=collate_batch)

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import time
from torchmetrics import F1Score

def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

f1_score = F1Score(task="binary", average='macro')
f1_score = f1_score.to(device)

def train_bert(model, train_loader, val_loader, optimizer, n_epochs):
  total_step = len(train_loader)
  BEST_F1 = 0
  for epoch in range(n_epochs):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    total_train_f1 = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      loss, prediction = model(pair_token_ids,
                             token_type_ids=seg_ids,
                             attention_mask=mask_ids,
                             labels=labels).values()

      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()

      total_train_loss += loss.item()
      total_train_acc  += acc.item()
      total_train_f1 += f1_score(torch.argmax(prediction, dim=1), labels)

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    train_f1 = total_train_f1/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      y_preds = torch.tensor(()).int().to(device)
      y_tests = torch.tensor(()).int().to(device)
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        loss, prediction = model(pair_token_ids,
                             token_type_ids=seg_ids,
                             attention_mask=mask_ids,
                             labels=labels).values()

        y_pred = torch.log_softmax(prediction, dim=1).argmax(dim=1)
        y_preds = torch.cat([y_preds, y_pred])
        y_tests = torch.cat([y_tests, labels])

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_f1 = f1_score(y_preds, y_tests)
    if val_f1 > BEST_F1:
        BEST_F1 = val_f1
        torch.save(model.state_dict(), '/content/drive/MyDrive/Tesis/click_model.pt')
    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_f1: {train_f1:.4f} | val_loss: {val_loss:.4f} val_f1: {val_f1:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train_bert(model, train_dataloader, valid_dataloader, optimizer, 3)

Epoch 1: train_loss: 0.4790 train_f1: 0.8633 | val_loss: 0.4332 val_f1: 0.8804
00:08:00.54
Epoch 2: train_loss: 0.3773 train_f1: 0.8909 | val_loss: 0.4303 val_f1: 0.8784
00:08:00.23
Epoch 3: train_loss: 0.2504 train_f1: 0.9328 | val_loss: 0.5008 val_f1: 0.8545
00:07:59.19
